In [ ]:
!pip install beautifulsoup4
!pip install newspaper3k
!pip install konlpy

In [ ]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab190912.sh
%cd ../

In [ ]:
!nvidia-smi

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()
mecab.morphs('하하하ㅏ하하ㅏㅎ하하')

# 1.BeautifulSoup 사용법

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
html = '''
<html>
    <head>
    </head>
    <body>
        <h1>장바구니
            <p id = 'clothes' class='name' title='라운드티'> 라운드티
                <span class = 'number'> 25 </span>
                <span class = 'price'> 29000 </span>
                <span class = 'menu'> 의류 </span>
                <a href = "http://www.naver.com">바로가기 </a>
            </p>
            <p id = 'watch' class='name' title='시계'> 시계
                <span class='number'> 28 </span>
                <span class='price'> 32000 </span>
                <span class='menu'> 악세사리 </span>
                <a href = "http://www.facebook.com"> 바로가기 </a>
            </p>
        </h1>
    </body>
</html>
'''

In [ ]:
# 객체 선언
soup = BeautifulSoup(html, 'html.parser')

## 원하는 정보만 뽑기

In [ ]:
print(soup.select('body'))

In [ ]:
print(soup.select('p'))

In [ ]:
print(soup.select('h1 .name .menu'))

In [ ]:
print(soup.select('html > body'))

## Newspaper3k 패키지

In [ ]:
from newspaper import Article

In [ ]:
url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002881076'

In [ ]:
article = Article(url, language='ko')
article.download()
article.parse()

In [ ]:
print('기사 제목 : ', article.title)
print()
print('기사 내용 \n', article.text)

## BeautifulSoup와 newspaper3k를 통해 크롤러 만들기

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def make_urllist(page_num, code, date):
    urllist = []
    for i in range(1, page_num + 1):
        url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
        news = requests.get(url, headers=headers)

        soup = BeautifulSoup(news.content, 'html.parser')

        # CASE 1
        news_list = soup.select('.newsflash_body .type06_headline li dl')

        # CASE 2
        news_list.extend(soup.select('.newsflash_body .type06 li dl'))

        # 각 뉴스로부터 a 태그인 <a href='주소'>에서 '주소'만을 가져옵니다.
        for line in news_list:
            urllist.append(line.a.get('href'))
    return urllist

In [ ]:
url_list = make_urllist(2, 101, 20200506) # 경제코드 101
print('뉴스 기사의 갯수 :', len(url_list))

In [ ]:
url_list[:5]

In [ ]:
idx2word = {'101':'경제', '102':'사회', '103':'생활/문화', '105':'IT/과학'}

In [ ]:
def make_data(urllist, code):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko') # 객체
        article.download()
        article.parse()
        text_list.append(article.text) # 기사

    # 데이터 프레임의 'news' 키 아래 파싱한 텍스트를 벨류로 붙여줍니다.
    df = pd.DataFrame({'news' : text_list})

    # 데이터 프레임의 'code'키 아래 한글 카테고리명을 붙여줍니다.
    df['code'] = idx2word[str(code)]
    return df

In [ ]:
data = make_data(url_list, 101)

In [ ]:
data.head()

## 데이터 수집 및 전처리

In [ ]:
code_list = [102, 103, 105]
code_list

In [ ]:
def make_total_data(page_num, code_list, date):
    df = None

    for code in code_list:
        url_list = make_urllist(page_num, code, date)
        df_temp = make_data(url_list, code)
        print(str(code) + '번 코드에 대한 데이터를 만들었습니다.')

        if df is not None:
            df = pd.concat([df, df_temp])
        else:
            df = df_temp
    return df

In [ ]:
df = make_total_data(1, code_list, 20200506)

In [ ]:
print('뉴스 기사의 갯수 :', len(df))

In [ ]:
df.sample(10)

In [ ]:
df = make_total_data(100, code_list, 20200506) # 대량의 크롤링 40~50분정도 걸립니다.

In [ ]:
len(df)

In [ ]:
# 해당 경로에 csv 저장하는 코드
csv_path = '/content/drive/MyDrive/Colab Notebooks/영우_5기_자연어/수업안/dataset/news_data.csv'
df.to_csv(csv_path, index=False)

In [ ]:
# 해당 경로에서 csv 읽어오는 코드
import pandas as pd
csv_path = 'news_data.csv'
df = pd.read_table(csv_path, sep=',')
print(len(df))
df.head()

In [ ]:
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

In [ ]:
print("결측치 검사 - ", df.isnull().sum())

In [ ]:
#중복 제거
print("기존 기사 갯수 : ", len(df))
df.drop_duplicates(subset=['news'], inplace=True)
print("제거 후 기사 갯수 : ", len(df))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "NanumGothic"

In [ ]:
df['code'].value_counts().plot(kind='bar')

In [ ]:
# 코드별로 그룹
df.groupby('code').size().reset_index(name='count')

## 불용어 제거

In [ ]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [ ]:
def preprocessing(data):
    text_data = []

    for sentence in data:
        temp_data = []
        temp_data = mecab.morphs(sentence)
        temp_data = [word for word in temp_data if not word in stopwords]
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))
    return text_data

In [ ]:
text_data = preprocessing(df['news'])

In [ ]:
print(text_data[0])

In [ ]:
len(text_data)

## 모델 학습 준비

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
df['code']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(text_data, df['code'], random_state=0)

In [ ]:
print('훈련용 뉴스 기사의 갯수 :', len(x_train))
print('테스트용 뉴스 기사의 갯수 : ', len(x_test))
print('훈련용 레이블의 갯수 :', len(y_train))
print('테스트용 레이블의 갯수 :', len(y_test))

In [ ]:
# DTM
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)

# TF-IDF 행렬
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [ ]:
print(x_train_tfidf)

In [ ]:
y_train

In [ ]:
# 모델 학습 (나이브베이즈 모델)
clf = MultinomialNB().fit(x_train_tfidf, y_train)

In [ ]:
def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [ ]:
new_sent = preprocessing(["민주당 일각에서 법사위의 체계·자구 심사 기능을 없애야 한다는 \
                           주장이 나오는데 대해 “체계·자구 심사가 법안 지연의 수단으로 \
                          쓰이는 것은 바람직하지 않다”면서도 “국회를 통과하는 법안 중 위헌\
                          법률이 1년에 10건 넘게 나온다. 그런데 체계·자구 심사까지 없애면 매우 위험하다”고 반박했다."           
])

In [ ]:
print(clf.predict(tfidf_vectorizer(new_sent)))

In [ ]:
new_sent = preprocessing(["20분기 연속으로 적자에 시달리는 LG전자가 브랜드 이름부터 성능, 디자인까지 대대적인 변화를 \
                          적용한 LG 벨벳은 등장 전부터 온라인 커뮤니티를 뜨겁게 달궜다. 사용자들은 “디자인이 예쁘다”, \
                          “슬림하다”는 반응을 보이며 LG 벨벳에 대한 기대감을 드러냈다."  
])

In [ ]:
print(clf.predict(tfidf_vectorizer(new_sent)))

In [ ]:
y_pred = clf.predict(tfidf_vectorizer(x_test))
print(metrics.classification_report(y_test, y_pred))

In [ ]:
# zero = input().split()
# print(zero)
one = ['박준영', '박유승', '민경태', '민재식', '류근용', '윤상영', '김인성', '정인성', '강성현', '팽성원', '고재원', '김동현', '강현', '이창영', '박승운', '박의찬', '강태균']
all = ['강성현', '강태균', '강현', '고재원', '김남우', '김동현', '김민규', '김인성', '류근용', '민경태', '박승운', '박유승', '박준영', '박진완', '손찬우', '윤상영', '이상원', '이창영', '정인성', '팽성원', '박의찬', '민재식']

nope = []
for person in all:
    if not person in one:
        nope.append(person)

print(nope)